<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />

# <center><font color= #1e8449 > <b>CU45_Planificación y promoción del destino en base a los patrones en origen de los turistas</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > III - Feature Engineering Domain </font>
***
> # <font color='steelblue'> <b>15.- Feature Importance</font>

<font color='steelblue'>Feature Importance is the process that assigns scores to the input characteristics to a model, which indicate the relative importance of each characteristic, in order, for example, to be able to select the most important ones.</font>


## <font color='steelblue'>Tasks</font>

<font color='steelblue'><b>Perform Feature importance from model coefficients</b>			
- Linear Regression Feature importance		
- Logistic Regression Feature importance		
    
<font color='steelblue'><b>Perform Feature importance from Decision Tree			
- CART Feature Importance		
- Random Forest Regression Feature Importance		
    
<font color='steelblue'><b>Perform Feature importance from Permutation testing</b>			
    
<font color='steelblue'><b>Evaluate a Logistic Regression model with feature selection</b>			

## Consideraciones casos CitizenLab programados en R

* Algunas de las tareas de este proceso se han realizado en los notebooks del proceso 05 Data Collection porque eran necesarias para las tareas ETL. En esos casos, en este notebook se referencia al notebook del proceso 05 correspondiente
* Otras tareas típicas de este proceso se realizan en los notebooks del dominio IV al ser más eficiente realizarlas en el propio pipeline de modelización.
* Por tanto en los notebooks de este proceso de manera general se incluyen las comprobaciones necesarias, y comentarios si procede
* Las tareas del proceso se van a aplicar solo a los archivos que forman parte del despliegue, ya que hay muchos archivos intermedios que no procede pasar por este proceso
* El nombre de archivo del notebook hace referencia al nombre de archivo del proceso 05 al que se aplica este proceso, por eso pueden no ser correlativa la numeración
* Las comprobaciones se van a realizar teniendo en cuenta que el lenguaje utilizado en el despliegue de este caso es R

## <font color='green'>File</font>

- <font color='steelblue'> <b>Input File: CU_45_08_03_turismo_receptor.csv</font>
- <font color='steelblue'> <b>Sampled Input File: CU_45_07_03_turismo_receptor.csv</font>
- <font color='steelblue'> <b>Output File: No aplica</font>

### Encoding

Con la siguiente expresión se evitan problemas con el encoding al ejecutar el notebook. Es posible que deba ser eliminada o adaptada a la máquina en la que se ejecute el código.

In [2]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

Warning message in Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8"):
“OS reports request to set locale to "es_ES.UTF-8" cannot be honored”


[1] ""

## <font color='green'>Settings</font>

### Libraries to use

In [3]:
library(readr)
library(dplyr)
library(tidyr)
library(forcats)
library(lubridate)



Attaching package: ‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




### Paths

In [4]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if using this option

In [5]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [6]:
iFile <- "CU_45_08_03_turismo_receptor.csv"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/CU_45_08_03_turismo_receptor.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [7]:
data <- read_csv(file_data)


Rows: 50294 Columns: 9
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): mes, pais_orig_cod, pais_orig, mun_dest, CMUN
dbl (3): mun_dest_cod, turistas, Target
lgl (1): is_train

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [8]:
data |> glimpse()

Rows: 50,294
Columns: 9
$ mes           <chr> "2019-08", "2021-07", "2021-07", "2022-01", "2019-08", "…
$ pais_orig_cod <chr> "110", "010", "010", "000", "128", "000", "011", "126", …
$ pais_orig     <chr> "Francia", "Total Europa", "Total Europa", "Total", "Rum…
$ mun_dest_cod  <dbl> 28161, 28176, 28132, 28141, 28130, 28126, 28075, 28005, …
$ mun_dest      <chr> "Valdemoro", "Villanueva de la Cañada", "San Martín de l…
$ turistas      <dbl> 466, 1375, 465, 54, 135, 30, 285, 768, 31, 1646, 116, 36…
$ CMUN          <chr> "161", "176", "132", "141", "130", "126", "075", "005", …
$ Target        <dbl> 466, 1375, 465, 54, 135, 30, 285, 768, 31, 1646, 116, 36…
$ is_train      <lgl> TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TRUE, TR…


Muestra de los primeros datos:

In [12]:
data |> slice_head(n = 5)

mes,pais_orig_cod,pais_orig,mun_dest_cod,mun_dest,turistas,CMUN,Target,is_train
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>
2019-08,110,Francia,28161,Valdemoro,466,161,466,TRUE
2021-07,010,Total Europa,28176,Villanueva de la Cañada,1375,176,1375,TRUE
2021-07,010,Total Europa,28132,San Martín de la Vega,465,132,465,TRUE
2022-01,000,Total,28141,Sevilla la Nueva,54,141,54,TRUE
2019-08,128,Rumania,28130,San Fernando de Henares,135,130,135,TRUE


## Feature importance from model coefficients

In [13]:
data <- data |>
    na.omit()

### Linear Regression Feature importance

In [14]:
# install the necessary packages if not already installed
if (!require(caret)) {
  install.packages('caret')
}

# load the necessary packages
library(caret)

# create the model
set.seed(123)
train_control <- trainControl(method = "cv", number = 5)
linear_model <- train(Target ~ ., data = data, method = "lm",
                      trControl = train_control)

# extract the coefficients
coef <- summary(linear_model$finalModel)$coefficients

# make a data frame of feature names and their corresponding coefficients
feature_importance <- data.frame(Feature = rownames(coef), Coefficient = coef[, "Estimate"])

# compute the absolute value of the coefficients
feature_importance$AbsCoefficient <- abs(feature_importance$Coefficient)

# sort the features by the absolute value of their coefficients in descending order
feature_importance <- feature_importance[order(-feature_importance$AbsCoefficient), ]

# print the feature importance
print(feature_importance)

Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice

Warning message in predict.lm(modelFit, newdata):
“prediction from a rank-deficient fit may be misleading”
Warning message in predict.lm(modelFit, newdata):
“prediction from a rank-deficient fit may be misleading”
Warning message in predict.lm(modelFit, newdata):
“prediction from a rank-deficient fit may be misleading”
Warning message in predict.lm(modelFit, newdata):
“prediction from a rank-deficient fit may be misleading”
Warning message in predict.lm(modelFit, newdata):
“prediction from a rank-deficient fit may be misleading”


                                                                        Feature
turistas                                                               turistas
(Intercept)                                                         (Intercept)
pais_orig_cod229                                               pais_orig_cod229
pais_orig_cod424                                               pais_orig_cod424
pais_orig_cod327                                               pais_orig_cod327
pais_orig_cod245                                               pais_orig_cod245
pais_orig_cod404                                               pais_orig_cod404
pais_orig_cod243                                               pais_orig_cod243
pais_orig_cod215                                               pais_orig_cod215
pais_orig_cod433                                               pais_orig_cod433
pais_orig_cod249                                               pais_orig_cod249
pais_orig_cod434                        

### Logistic Regression Feature importance

No aplica

## Decision Tree Feature Importance

### CART Regression Feature Importance

In [15]:
# install the necessary packages if not already installed
if (!require(rpart)) {
  install.packages('rpart')
}

# load the necessary packages
library(rpart)

# fit the model
cart_model <- rpart(Target ~ ., data = data, method = "anova")

# compute feature importances
importance <- as.data.frame(varImp(cart_model))

# sort by importance
importance <- importance[order(-importance$Overall), ]

# print the feature importance
print(importance)

Loading required package: rpart



[1] 4.27540375 0.92840868 0.91951914 0.77648123 0.21746804 0.21746804 0.02441950
[8] 0.01244799


### CART Classification Feature Importance

No aplica

### Random Forest Regression Feature Importance

ATENCIÓN: Este proceso es costoso en recursos.

In [9]:
# install the necessary packages if not already installed
if (!require(randomForest)) {
  install.packages('randomForest')
}

# load the necessary packages
library(randomForest)

# fit the model
rf_model <- randomForest(Target ~ ., data = data, importance = TRUE)

# get feature importances
importance <- rf_model$importance

# make a data frame of feature names and their corresponding importances
feature_importance <- data.frame(Feature = rownames(importance), Importance = importance[,1]) # Adjust the index based on your requirement

# sort the features by importance in descending order
feature_importance <- feature_importance[order(-feature_importance$Importance), ]

# print the feature importance
print(feature_importance)

                    Feature   Importance
mun_dest_cod   mun_dest_cod 346461864.76
CMUN                   CMUN 282898525.92
mun_dest           mun_dest 212597478.48
pais_orig_cod pais_orig_cod 166564554.68
turistas           turistas 119379733.50
pais_orig         pais_orig  32186340.64
mes                     mes   3750848.44
is_train           is_train     77949.02


In [10]:
rf_model$importance

,%IncMSE,IncNodePurity
mes,3750848.44,1.142556e+11
pais_orig_cod,166564554.68,6.109093e+11
pais_orig,32186340.64,2.365744e+11
mun_dest_cod,346461864.76,1.922399e+11
mun_dest,212597478.48,2.086304e+11
turistas,119379733.50,4.089973e+12
CMUN,282898525.92,1.714569e+11
is_train,77949.02,8.536193e+09


### Random Forest Classification Feature Importance

No aplica

## Permutation Feature Importance

### Permutation Feature Importance for Regression

In [11]:
# install the necessary packages if not already installed
if (!require(ranger)) {
  install.packages('ranger')
}

# load the necessary packages
library(ranger)

# fit the model
rf_model <- ranger(Target ~ ., data = data, importance = 'permutation')

# get feature importances
importance <- data.frame(Feature = names(rf_model$variable.importance), Importance = rf_model$variable.importance)

# sort the features by importance in descending order
importance <- importance[order(-importance$Importance), ]

# print the feature importance
print(importance)

Loading required package: ranger


Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance




                    Feature  Importance
mun_dest_cod   mun_dest_cod 271146955.1
CMUN                   CMUN 258862935.3
mun_dest           mun_dest 215161355.5
pais_orig_cod pais_orig_cod 197744889.8
turistas           turistas 119085138.0
pais_orig         pais_orig  37156365.6
mes                     mes   3818209.9
is_train           is_train    187336.7


### Permutation Feature Importance for Classification

In [ ]:
No aplica

## Evaluating a Logistic Regression model with feature selection.

### Evaluating with all selected features.

### Evaluating with feature selection performed using feature importance.

<font color='tomato'>Select type of Feature Importance to use</font>

<font color='darkgrey'><b>Operation</font>